In [2]:
import matplotlib
matplotlib.use("Agg")

import os
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers.core import Dense
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# инициализируем данные и метки
data = []
labels = []
train_labels = os.listdir('train')
train_labels.sort()

# цикл по изображениям из папки train - вытаскиваем картинки и кладем в data, их класс (имя симпсона) в labels
for training_name in train_labels:
    dir = os.path.join('train', training_name)
    current_label = training_name

    path = 'train/' + training_name
    x = os.listdir(dir)

    for y in x:
        image = cv2.imread(path + "/" + str(y))
        image = cv2.resize(image, (32, 32)).flatten()
        data.append(image)
        labels.append(current_label)

# масштабируем интенсивности пикселей в диапазон [0, 1]
data = np.array(data, dtype="float") / float(255)
labels = np.array(labels)

# разбиваем данные на обучающую и тестовую выборки
seed = 50 # random.randint(1, 100)
print("seed =", seed)
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2, random_state=seed)

# конвертируем метки из целых чисел в векторы
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

# определим архитектуру 3072-1024-512-3 с помощью Keras
model = Sequential()
model.add(Dense(1024, input_shape=(3072,), activation="sigmoid"))
model.add(Dense(512, activation="sigmoid"))
model.add(Dense(len(lb.classes_), activation="softmax"))

# компилируем модель, используя Adam как оптимизатор и категориальную кросс-энтропию в качестве функции потерь
opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
# архитектура нейронки
model.summary()

# обучаем нейросеть
H = model.fit(trainX, trainY, validation_data=(testX, testY), epochs=100, batch_size=32)

# оцениваем нейросеть
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=lb.classes_))

# строим графики потерь и точности
#plt.style.use("plot")
N = np.arange(0, 100)
#plt.figure()
plt.plot(N, H.history["loss"], label="training loss")
plt.plot(N, H.history["val_loss"], label="validation loss")
plt.plot(N, H.history["accuracy"], label="training accuracy")
plt.plot(N, H.history["val_accuracy"], label="validation accuracy")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig('plot_forw.png')
plt.show()

# начинаем предсказывать принадлежность к классу
test_labels = os.listdir('test')
test_labels.sort()

# цикл по изображениям test
for test_file in test_labels:
    # загружаем входное изображение и меняем его размер на необходимый
    path_for_save = 'result/' + test_file
    test_file = 'test/' + test_file
    image = cv2.imread(test_file)
    image_copy = image.copy()
    image = cv2.resize(image, (32, 32)).flatten()
    # масштабируем значения пикселей к диапазону [0, 1]
    image = image.astype("float") / 255.
    image = image.reshape((1, image.shape[0]))

    # массив предсказания класса картинки
    predictions = model.predict(image)
    # находим индекс класса с наибольшей вероятностью
    i = predictions.argmax(axis=1)[0]
    label = lb.classes_[i]

    # выводим название класса и вероятность принадлежности на картинку
    text = f'{label}: {round(predictions[0][i] * 100, 2)}%'
    cv2.putText(image_copy, text, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (250, 250, 255), 2)
    plt.imshow(cv2.cvtColor(image_copy, cv2.COLOR_BGR2RGB))
    plt.show()
    cv2.imwrite(path_for_save, image_copy)


seed = 50
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 1024)              3146752   
_________________________________________________________________
dense_4 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 1539      
Total params: 3,673,091
Trainable params: 3,673,091
Non-trainable params: 0
_________________________________________________________________


C:\Users\Alefskii\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/100
10/10 [==============================] - 2s 85ms/step - loss: 1.5022 - accuracy: 0.4050 - val_loss: 0.8549 - val_accuracy: 0.6667
Epoch 2/100
10/10 [==============================] - 0s 33ms/step - loss: 1.0958 - accuracy: 0.5485 - val_loss: 0.9455 - val_accuracy: 0.6667
Epoch 3/100
10/10 [==============================] - 0s 36ms/step - loss: 1.0150 - accuracy: 0.5398 - val_loss: 0.8821 - val_accuracy: 0.6667
Epoch 4/100
10/10 [==============================] - 0s 41ms/step - loss: 0.8738 - accuracy: 0.6373 - val_loss: 1.1854 - val_accuracy: 0.2533
Epoch 5/100
10/10 [==============================] - 0s 33ms/step - loss: 0.9934 - accuracy: 0.4512 - val_loss: 0.8870 - val_accuracy: 0.6667
Epoch 6/100
10/10 [==============================] - 0s 32ms/step - loss: 0.9167 - accuracy: 0.4956 - val_loss: 0.8715 - val_accuracy: 0.6667
Epoch 7/100
10/10 [==============================] - 0s 33ms/step - loss: 0.8087 - accuracy: 0.6250 - val_loss: 0.7953 - val_accuracy: 0.6533
Epoch 

10/10 [==============================] - 0s 31ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.6856 - val_accuracy: 0.8000
Epoch 59/100
10/10 [==============================] - 0s 34ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.6793 - val_accuracy: 0.8000
Epoch 60/100
10/10 [==============================] - 0s 32ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.6821 - val_accuracy: 0.8000
Epoch 61/100
10/10 [==============================] - 0s 31ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.6954 - val_accuracy: 0.8000
Epoch 62/100
10/10 [==============================] - 0s 33ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.6859 - val_accuracy: 0.7867
Epoch 63/100
10/10 [==============================] - 0s 37ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.6959 - val_accuracy: 0.8000
Epoch 64/100
10/10 [==============================] - 0s 33ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.6901 - val_accuracy: 0.7867
Epoch 65/100

<ipython-input-2-e3b9932dafc0>:82: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
<ipython-input-2-e3b9932dafc0>:110: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
